# Exercise 0 - PyTorch Tutorial
## Yandex Data Science School RL Course 2019

In this exercise you will get up to speed with pyTorch a modern dynamic deep learning framework by Facebook.

We will use it for the rest of the book as our platform for training Deep Neural Network.

Plenty of tutorials are available online. For example the official one are here https://pytorch.org/tutorials/#getting-started and the official documentations https://pytorch.org/docs/stable/index.html

## Part 1. Simple Graph
Create a DNN with the following layers:
1. Linear `n_input` to 5
2. Relu
3. Linear 5 to 20
4. Relu
5. Linear 20 to `num_classes`
5. Dropout with probability `dropout_prob`
6. soft max

hints:
1.use `nn.Sequential` to create a cascade of layers
2. use `nn.X` to use layer of type `X`

In [1]:
import torch
import torch.nn as nn

class OurModule(nn.Module):
    def __init__(self, num_inputs, num_classes, dropout_prob=0.3):
        super(OurModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs, 5),
            nn.ReLU(True),

            nn.Linear(5, 20),
            nn.ReLU(True),
            nn.Linear(20, num_classes),
            nn.Dropout(dropout_prob),
            nn.Softmax(1)
        )


    def forward(self, x):
        return self.pipe(x)

if __name__ == "__main__":
    net = OurModule(num_inputs=2, num_classes=3)
    print(net)
    v = torch.FloatTensor([[2, 3]])
    out = net(v)
    print(out)

OurModule(
  (pipe): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=5, out_features=20, bias=True)
    (3): ReLU(inplace)
    (4): Linear(in_features=20, out_features=3, bias=True)
    (5): Dropout(p=0.3)
    (6): Softmax()
  )
)
tensor([[0.3396, 0.3325, 0.3278]], grad_fn=<SoftmaxBackward>)


## Part 2. Using GPU in Colab
Go to Runtime menu and select "change runtime type" and checkbox GPU.
Run the following code to test if you have succeeded.

```
print("Cuda's availability is %s" % torch.cuda.is_available())
```

In [2]:
print("Cuda's availability is %s" % torch.cuda.is_available())

Cuda's availability is True


In order to copy a tensor to the device you can user `.to('cuda')` or just `.cuda`

In [3]:
import torch
v = torch.FloatTensor([[2, 3]])
print(v.to('cuda'))
print(v.cuda())

tensor([[2., 3.]], device='cuda:0')
tensor([[2., 3.]], device='cuda:0')


## Part 3. Using TensorBoard in Colab

Note: Ngrock is not necessary for someone who runs the notebook locally. If you run locally you can skip the ngrock issue.

We would like to track learning. For this we have handy TensorBoard GUI.=

If we work in Colab, since tensorboard is a server-client application, we need to create a network tunnel to access it through the colab virtual machine. See detailed explanation here https://medium.com/@tommytao_54597/use-tensorboard-in-google-colab-16b4bb9812a6 

Use the following lines to use tensorboard in colab:

```
!pip install tensorboardX
from tensorboardcolab import TensorBoardColab
tbc=TensorBoardColab(graph_path='./runs')
```
browse the printed url to access the tensorboard


In [7]:
LOG_DIR = './log'

!pip install tensorboardX
from tensorboardcolab import TensorBoardColab
tbc=TensorBoardColab(graph_path=LOG_DIR)


Wait for 8 seconds...
TensorBoard link:
https://acf2d6ab.ngrok.io


use tensorboardX SummaryWritter to create a write to the ./log/demo folder as log_dir.
user the writter to log a sin cos and than values at each angle between -360 and 360.
Visit the ngrok url of the TensorBoard to see your plots.
It should look something like:

![alt text](https://i.ibb.co/rGbpc5r/Screen-Shot-2019-03-22-at-14-09-30.png)

In [0]:
import math
from tensorboardX import SummaryWriter


if __name__ == "__main__":
    writer = SummaryWriter(logdir=LOG_DIR)

    funcs = {"sin": math.sin, "cos": math.cos, "tan": math.tan}

    for angle in range(-360, 360):
        angle_rad = angle * math.pi / 180
        for name, fun in funcs.items():
            val = fun(angle_rad)

            writer.add_scalar(name, val)
    writer.close()
    

## Part 4. MNIST Hello World

1. Implement `Net` class below to create the following architecture:

  1. Relu Conv2D 20x20x5 in_channels, out_channels, kernel_size, stride = 1,20,5,1
  2. Max Pooled2D kernel_size, stride=2,2
  3. Relu Conv2D in_channels, out_channels, kernel_size, stride = 20,50,5,1
  4. Max Pooled2D kernel_size, stride=2,2
  5. Fully connected in_features, out_features = 800, 500 (you will need to flat the tensor first)
  6. Fully connected in_features, out_features = 500,10
  
  You can use the help class `Flatten`

2. You can use the `get_loader` function to create a generator of batches.
3. Don't forget to move the network to the gpu to speed up traning (`.to('cuda')`)
4. Use SGD optimizer `optim.SGD`
5. Use the following default parameters, you can use `get_args` to get an object with the default params, you will need the first line to make it work. https://github.com/spyder-ide/spyder/issues/3883
  * batch_size = 65
  * test_batch_size = 1000
  * epochs = 10
  * learning_rate = 0.01
  * momentum = 0.5
6. Implement a `train` and `test` function with the  signature below
7. Use the writer=SummaryWriter to log training and test loss as well as test accuracy over time.
8. Use the main method below.


In [0]:
import sys; sys.argv=['']; del sys # without this argparse won't work properly. https://github.com/spyder-ide/spyder/issues/3883 


class Flatten(nn.Module):
  def __init__(self):
    super().__init__()
  def forward(self, input):
      return input.view(input.size(0), -1)

class Net(nn.Module):
  def __init__(self):
      super(Net, self).__init__()
      self.net = None

  def forward(self, x):
      return self.net(x)
    
def get_loader(train, batch_size, use_cuda):
  kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

  loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', 
                   train=train, 
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, 
      shuffle=True, 
      **kwargs)
  return loader

def train(model, device, train_loader, optimizer, epoch, writer):
  raise NotImplemented
  
def test(model, device, test_loader, epoch, writer):
  raise NotImplemented
  
def get_args():
  parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
  parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                      help='input batch size for training (default: 64)')
  parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                      help='input batch size for testing (default: 1000)')
  parser.add_argument('--epochs', type=int, default=10, metavar='N',
                      help='number of epochs to train (default: 10)')
  parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                      help='learning rate (default: 0.01)')
  parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                      help='SGD momentum (default: 0.5)')
  parser.add_argument('--no-cuda', action='store_true', default=False,
                      help='disables CUDA training')
  parser.add_argument('--seed', type=int, default=1, metavar='S',
                      help='random seed (default: 1)')
  parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                      help='how many batches to wait before logging training status')

  parser.add_argument('--save-model', action='store_true', default=False,
                      help='For Saving the current Model')
  args = parser.parse_args()
  return args

def main():
    args = get_args()
    use_cuda = torch.cuda.is_available()
    print('use_cuda = {}'.format(use_cuda))
    device = torch.device("cuda" if use_cuda else "cpu")

    torch.manual_seed(args.seed)
    now = datetime.now()
    writer = SummaryWriter(log_dir=LOG_DIR + '/nminst_' + now.strftime("%Y%m%d-%H%M%S"))

    train_loader = get_loader(train=True, batch_size=args.batch_size, use_cuda=use_cuda)
    test_loader = get_loader(train=False, batch_size=args.test_batch_size, use_cuda=use_cuda)
    model = Net().to(device)
    optimizer = # ADD THIS
    
    for epoch in range(1, args.epochs + 1):
        print('epoch={}'.format(epoch))
        train(model, device, train_loader, optimizer, epoch, writer)
        test(model, device, test_loader, epoch, writer)
